### QSVM4EO

This notebook will present the an example of using quantum-enhanced support vector machines (QSVM) for classification tasks on multi-spectral Earth Observation (EO) data. The main topics covered are introduction to classical SVM, where quantum computation could be used in the SVM calculations, the intricacies of preparing and encoding the classical data into useful quantum states and training QSVMs on gate-based quantum software simulators. Finally, some results on comparing classical and quantum-enhanced SVM models will be presented.

QSVM code is adapted from Qiskit QSVM example: https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html

In [1]:
import qiskit

from qiskit_machine_learning.utils import algorithm_globals

algorithm_globals.random_seed = 12345

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def read_csv(file_dir, selected_columns):
    df = pd.read_csv(file_dir)
    df_selected = df[selected_columns]
    X = df_selected.values
    Y = df['Label'].values
    return X,Y

In [3]:
train_data = 'data/train_32.csv'
test_data = 'data/test_32.csv'

# four_features_data = ['B02', 'B03', 'B04', 'B08']
# adhoc_dimension = 4
eight_features_data = ['B02', 'B03', 'B04', 'B08', 'NDVI', 'EVI', 'SAVI', 'NDWI']
adhoc_dimension = 8


In [4]:
train_features, train_labels = read_csv(train_data, eight_features_data)

test_features, test_labels = read_csv(test_data, eight_features_data)

# scaler = StandardScaler().fit(train_features)
# X_train_scaled = scaler.transform(train_features)
# X_test_scaled = scaler.transform(test_features)

In [5]:
from sklearn.svm import SVC

svc_classifier = SVC(kernel='linear')
svc_classifier.fit(train_features, train_labels)

train_score = svc_classifier.score(train_features, train_labels)
test_score = svc_classifier.score(test_features, test_labels)


print(f"Precomputed classical kernel classification test score: {test_score}")



Precomputed classical kernel classification test score: 0.5419921875


In [6]:
scaler = StandardScaler().fit(train_features)
train_features_scaled = scaler.transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [7]:
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel, FidelityStatevectorKernel

adhoc_feature_map = ZZFeatureMap(feature_dimension=adhoc_dimension, reps=1, entanglement="full")
#adhoc_feature_map = ZFeatureMap(feature_dimension=adhoc_dimension, reps=2)

#sampler = Sampler()

#fidelity = ComputeUncompute(sampler=sampler)

# adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)
adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)

adhoc_kernel.feature_map.decompose().draw()


┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     ├───┤├─────────────┤                                          └───┘»
q_3: ┤ H ├┤ P(2.0*x[3]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_4: ┤ H ├┤ P(2.0*x[4]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_5: ┤ H ├┤ P(2.0*x[5]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_6: ┤ H ├┤ P(2.0*x[6]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_7: ┤ H ├┤ P(2.0*x[7]) ├───────────────────────────────────────────────»
     └───┘└─────────────┘                                               »
«                                                    »
«q_0: ──────────────────────────────────■─────────■──»
«                                       │         │  »
«q_1: ──────────────────────────────────┼────■────┼──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐  │  »
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├──┼──»
«     └──────────────────────────────┘└───┘└───┘┌─┴─┐»
«q_3: ──────────────────────────────────────────┤ X ├»
«                                               └───┘»
«q_4: ───────────────────────────────────────────────»
«                                                    »
«q_5: ───────────────────────────────────────────────»
«                                                    »
«q_6: ───────────────────────────────────────────────»
«                                                    »
«q_7: ───────────────────────────────────────────────»
«                                                    »
«                                                         »
«q_0: ───────────────────────────────────────■─────────■──»
«                                            │         │  »
«q_1: ──────────────────────────────────■────┼────■────┼──»
«     ┌──────────────────────────────┐┌─┴─┐  │    │    │  »
«q_2: ┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├──┼────┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐┌─┴─┐  │  »
«q_3: ┤ P(2.0*(π - x[0])*(π - x[3])) ├─────┤ X ├┤ X ├──┼──»
«     └──────────────────────────────┘     └───┘└───┘┌─┴─┐»
«q_4: ───────────────────────────────────────────────┤ X ├»
«                                                    └───┘»
«q_5: ────────────────────────────────────────────────────»
«                                                         »
«q_6: ────────────────────────────────────────────────────»
«                                                         »
«q_7: ────────────────────────────────────────────────────»
«                                                         »
«                                                              »
«q_0: ───────────────────────────────────────■──────────────■──»
«                                            │              │  »
«q_1: ──────────────────────────────────■────┼─────────■────┼──»
«                                       │    │         │    │  »
«q_2: ──────────────────────────────────┼────┼────■────┼────┼──»
«     ┌──────────────────────────────┐┌─┴─┐  │  ┌─┴─┐  │    │  »
«q_3: ┤ P(2.0*(π - x[1])*(π - x[3])) ├┤ X ├──┼──┤ X ├──┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐└───┘┌─┴─┐  │  »
«q_4: ┤ P(2.0*(π - x[0])*(π - x[4])) ├─────┤ X ├─────┤ X ├──┼──»
«     └──────────────────────────────┘     └───┘     └───┘┌─┴─┐»
«q_5: ────────────────────────────────────────────────────┤ X ├»
«           

In [12]:
adhoc_matrix_train = adhoc_kernel.evaluate(x_vec=train_features)
adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=test_features, y_vec=train_features)

In [13]:
adhoc_svc = SVC(kernel="precomputed")

adhoc_svc.fit(adhoc_matrix_train, train_labels)

adhoc_score_precomputed_kernel = adhoc_svc.score(adhoc_matrix_test, test_labels)

print(f"Precomputed quantum kernel classification test score: {adhoc_score_precomputed_kernel}")

Precomputed quantum kernel classification test score: 0.330078125


In [14]:
matrix_train = adhoc_kernel.evaluate(x_vec=train_features_scaled)
adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=test_features_scaled, y_vec=train_features_scaled)

In [15]:
adhoc_svc = SVC(kernel="precomputed")

adhoc_svc.fit(adhoc_matrix_train, train_labels)

adhoc_score_precomputed_kernel = adhoc_svc.score(adhoc_matrix_test, test_labels)

print(f"Precomputed quantum kernel classification test score: {adhoc_score_precomputed_kernel}")

Precomputed quantum kernel classification test score: 0.4990234375
